## Importing

In [1]:
import sys, os

module_path = os.getcwd() + '/fmri'

if (sys.path.count(module_path) == 0):
    sys.path.append(os.path.abspath(os.path.join('..', '..', 'fmri/fmri/')))
    
import load_data as ld
import dataset_utilities as du
import numpy as np

/usr/lib/python2.7/dist-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/usr/lib/python2.7/dist-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


Let's get some preprocessed data from the 2010 movie dataset.  The load_data module uses multiple methods from the fmri_preprocessing module plus parallelization accross subjects to quickly load in the 2010 Wagner movie dataset.

In [2]:
ds_list = ld.get_2010_preprocessed_data(num_subjects=5, mask_path='../masks/aal_l_hippocampus_3x3x3.nii', n_cpu=5, combine=False)

 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
Finished subject: 4
Finished subject: 3
Finished subject: 2
Finished subject: 0
Finished subject: 1


Unfortunately there are some pesky warnings that I could not for the life of me get to go away. Ignorning these, we can take a look at what we got from importing the data. 

In [3]:
print("Gathered Datasets for {0} subjects".format(len(ds_list)))
print("Shape of each Dataset: {0}".format(ds_list[0].shape))

Gathered Datasets for 5 subjects
Shape of each Dataset: (633, 318)


So we have a list of 5 subjects Dataset objects.  Each Dataset contains the preprocessed and spliced runs with 633 time points and 318 voxels inside of the left hippocampus mask we supplied.

Let's take just one of the Datasets and examine the time series for the voxel 50.

In [4]:
ds = ds_list[0]
du.voxel_plot(ds, 50)

This voxel_plot() function just quickly lets us examine the time series for a single voxel.  You can see that the runs of have been detrended and z-scored.  Now lets do something a little more interesting.
***

## Searchlights

In order to run a searchlight analysis between multiple subjects, we need to combine the datasets into one single
Dataset object to feed into pymvpa's searchlight function.  To do this we have two options:

1: You already knew from the start that you wanted the datasets combined, so in the call to load_data we set the combine keyword to True (which it is by default)

In [5]:
combined_ds = ld.get_2010_preprocessed_data(num_subjects=5, mask_path='../masks/aal_l_hippocampus_3x3x3.nii', n_cpu=5, combine=True)

 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
Finished subject: 1
Finished subject: 2
Finished subject: 0
Finished subject: 4
Finished subject: 3


2: We take the dataset list we got earlier and use the following function to combine them

In [6]:
cds = du.combine_datasets(ds_list)

Either way we get the same single Dataset object that is ready to be passed to a searchlight method.

In [7]:
print(combined_ds.shape)
print(cds.shape)

(5, 318, 633)
(5, 318, 633)


Let's run a simple Pearson's correlation first, with single voxel univariate comparisons.  We do this by setting the
searchlight radius to 0.

In [8]:
results = du.run_searchlight(cds, metric='correlation', radius=0, n_cpu=4)
print(results.__class__)
print(results.shape)

<class 'mvpa2.datasets.base.Dataset'>
(1, 318)


So here we got a Dataset object with 318 average Pearson's correlation values for all pairwise combinations of subjects.  From here we can easily export the data to a nifti file and look at the results in AFNI or another fmri viewing tool.  A simple function to do this is to uncomment and run the following:

In [9]:
#du.export_to_nifti(results, 'left_hippo_corr_r0')   

Now to actually make use of the searchlight functionality we can increase the radius.  This will take the regional average in each searchlight and compare each subjects timeseries, and return an average at each voxel.

In [10]:
corr_res = du.run_searchlight(cds, metric='correlation', radius=2, n_cpu=4)
print(corr_res.shape)

(1, 318)


You can change the measurement to canonical correlation analysis by simply using metric='cca'.

In [11]:
cca_res = du.run_searchlight(cds, metric='cca', radius=2, n_cpu=20)

Then we can get a quick look at the how these two results compare in a graph by doing the following:

In [12]:
du.plot_isc_vs_isi(corr_res, cca_res, 'Example Title', save=False, filename=None)

/usr/lib/pymodules/python2.7/matplotlib/collections.py:548: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


### Event-based analysis

When you have timeseries data that you want to treat as seperate events, there is one main step you have to take before using many of the methods I have created.  Given a dataset, say the fusiform of 5 subjects:

In [13]:
cds = ld.get_2010_preprocessed_data(num_subjects=5, mask_path='../masks/aal_l_fusiform_3x3x3.nii', n_cpu=5)
print(cds.shape)

 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
 * Please note: warnings are printed only once, but underlying problem might occur many times *
Finished subject: 2
Finished subject: 1
Finished subject: 4
Finished subject: 0
Finished subject: 3
(5, 768, 633)


We can now do add in event boundaries, where we wish to average sequences of time together in order to do various other analyses.  These boundaries go into a Dataset variable called "event_bounds".

In [14]:
cds.a["event_bounds"] = [0,100,200,300,400,500,600]

Let's visualize the hierachrical clustering dendrogram that uses Pearson's correlation as the similarity metric.  We do this simply by using the following code:

In [15]:
du.create_dendrogram(cds)

Now lets do a Supper Vector Machine (SVM) complete cross validation within a searchlight of radius 1.  This method takes a dataset that has event boundaries and completes an SVM classification based on N subjects (5 in our case).  It trains the classifier on N-1 subjects (4 for us) and then tests the models accuracy on the final held out subject.  It does this using all subjects as the testing data, and then returns the average accuracy between all of the cross-validation tests.  This scalar average is returned as the center of the searchlight.  Here is all you need to do:

In [16]:
svm_results = du.run_searchlight(cds, metric="scene_svm_cv", radius=1, n_cpu=40)
print(svm_results.shape)

(1, 768)


So we have the results of SVM.  Lets analyze it. 

In [17]:
average_class_accuracy = np.average(svm_results)
print(average_class_accuracy)

0.294053819444


So we have the average classification accuracy of the SVM.  But we need to remember that we only had 6 events
([0,100], [100,200], [200,300], [300,400], [400,500], [500,633]), so by chance the classifier will already have average accuracy of 1/6 = 0.16667.

If we wanted to take this a step further, and see which portions of the brain are best at classifying which events, we can use the finer grain searchlight method which returns (flattened) confusion matrices instead of a scalar average. 

In [23]:
import measures
reload(measures)
reload(du)
svm_results = du.run_searchlight(cds, metric="event_svm_cv_cm", radius=1, n_cpu=40)
print(svm_results.shape)

/usr/local/lib/python2.7/dist-packages/numpy/lib/utils.py:99: DeprecationWarning: `chisqprob` is deprecated!
stats.chisqprob is deprecated in scipy 0.17.0; use stats.distributions.chi2.sf instead.
  warnings.warn(depdoc, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/numpy/lib/utils.py:99: DeprecationWarning: `chisqprob` is deprecated!
stats.chisqprob is deprecated in scipy 0.17.0; use stats.distributions.chi2.sf instead.
  warnings.warn(depdoc, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/numpy/lib/utils.py:99: DeprecationWarning: `chisqprob` is deprecated!
stats.chisqprob is deprecated in scipy 0.17.0; use stats.distributions.chi2.sf instead.
  warnings.warn(depdoc, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/numpy/lib/utils.py:99: DeprecationWarning: `chisqprob` is deprecated!
stats.chisqprob is deprecated in scipy 0.17.0; use stats.distributions.chi2.sf instead.
  warnings.warn(depdoc, DeprecationWarning)
/usr/lib/python2.7/dist-packages/mvp

(36, 768)


Ignoring all of the deprecation warnings, we see that we have gleaned an array of 36 values for each of the 768 voxels.  These 36 values correspond to a 6x6 confusion matrix, which we can return to its original shape by simply reshaping it

In [25]:
conf_mat_array = svm_results.samples.T.reshape((768,6,6))

In [30]:
print(conf_mat_array[0:2]) # here are the first 2

[[[0 1 0 0 1 0]
  [1 0 0 3 0 0]
  [2 0 3 1 3 0]
  [0 3 0 0 1 0]
  [1 0 1 0 0 0]
  [1 1 1 1 0 5]]

 [[0 2 0 0 2 0]
  [1 0 0 2 0 0]
  [2 0 3 1 2 0]
  [0 1 0 1 1 0]
  [1 0 1 0 0 0]
  [1 2 1 1 0 5]]]


As you can see we now have a bunch of confusion matricies.  To get the accuracy for each event on each confusion array, we can call a convenience method in the dataset_utilities module.

In [27]:
accuracies = du.confusion_matrix_accuracies(conf_mat_array)

In [29]:
print(accuracies[0:2]) # here are the first 2

[array([ 0. ,  0. ,  0.6,  0. ,  0. ,  1. ]), array([ 0. ,  0. ,  0.6,  0.2,  0. ,  1. ])]


From here we can average all of these accuracies to get the mean accuracy accross all voxels, or we could export each individual event to a nifti file or any other number of analyses.

In [33]:
accuracies = np.array(accuracies)
avg_event_acc = np.mean(accuracies, axis=0)
print(avg_event_acc)

# Or export to nifti for a certain event.  Say the second event
second_event = accuracies[:,1].reshape((1,accuracies.shape[0]))
#du.export_to_nifti(second_event, "some title")

[ 0.14765625  0.15651042  0.18072917  0.17265625  0.10729167  0.99947917]


Wow, looking at the data at this finer grain level we can see that the final event was classified correctly almost every single time, while all other events were essentially inseperable from each other.  Good thing we took a closer look.

### Moving forward

There are many other methods that are useful for event-based and continuous (well psuedo-continuous) multivariate and univariate pattern analyses in this toolkit.  Take a look at the methods in measures.py and see how easy it is to add your own.  The event based stuff we did above easily expands to cover scene based analysis once you have the scenes boundaries created by RAs.  I did a fair amount of work looking into this with Matchstick Men.  Take a look at scene_keys.txt inside the scene_changes folder.  

Goodluck, and if there are any questions on these methods just message me on slack @zpeugh.